> Below is the H and E normalization code

In [1]:
import numpy as np
import cv2
import os
"""
            Input: RGB image
            Step 1: Convert RGB to OD
            Step 2: Remove data with OD intensity less than β
            Step 3: Calculate  singular value decomposition (SVD) on the OD tuples
            Step 4: Create plane from the SVD directions corresponding to the
            two largest singular values
            Step 5: Project data onto the plane, and normalize to unit length
            Step 6: Calculate angle of each point wrt the first SVD direction
            Step 7: Find robust extremes (αth and (100−α)th 7 percentiles) of the
            angle
            Step 8: Convert extreme values back to OD space

            Output: Optimal Stain Vectors

"""

def normalize_he_images(input_dir, output_dir, Io=240, alpha=1, beta=0.15):
    # Create output directories
    os.makedirs(output_dir, exist_ok=True)
    h_folder = os.path.join(output_dir, "H-stains")
    e_folder = os.path.join(output_dir, "E-stains")
    os.makedirs(h_folder, exist_ok=True)
    os.makedirs(e_folder, exist_ok=True)

    for filename in os.listdir(input_dir):
        if filename.endswith((".jpg", ".png", ".tif", ".bmp")):
            img_path = os.path.join(input_dir, filename)
            ############### INPUT RGB IMAGE #######################
            #Using opencv to read images may bemore robust compared to using skimage
            #but need to remember to convert BGR to RGB.
            #Also, convert to float later on and normalize to between 0 and 1.
            try: 
                img=cv2.imread(img_path, 1)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                #Io = 240 # Transmitted light intensity, Normalizing factor for image intensities
                #alpha = 1  #As recommend in the paper. tolerance for the pseudo-min and pseudo-max (default: 1)
                #beta = 0.15 #As recommended in the paper. OD threshold for transparent pixels (default: 0.15)


                ######## Step 1: Convert RGB to OD ###################
                ## reference H&E OD matrix.
                #Can be updated if you know the best values for your image. 
                #Otherwise use the following default values. 
                #Read the above referenced papers on this topic. 
                HERef = np.array([[0.5626, 0.2159],
                                [0.7201, 0.8012],
                                [0.4062, 0.5581]])
                ### reference maximum stain concentrations for H&E
                maxCRef = np.array([1.9705, 1.0308])


                # extract the height, width and num of channels of image
                h, w, c = img.shape

                # reshape image to multiple rows and 3 columns.
                #Num of rows depends on the image size (wxh)
                img = img.reshape((-1,3))

                # calculate optical density
                # OD = −log10(I)  
                #OD = -np.log10(img+0.004)  #Use this when reading images with skimage
                #Adding 0.004 just to avoid log of zero. 

                OD = -np.log10((img.astype(np.float64)+1)/Io) #Use this for opencv imread
                #Add 1 in case any pixels in the image have a value of 0 (log 0 is indeterminate)

                """
                from mpl_toolkits.mplot3d import Axes3D

                fig = plt.figure(figsize=(16, 8))
                ax1 = fig.add_subplot(121, projection='3d')
                ax1.scatter(img[:,0],img[:,1],img[:,2])
                ax2 = fig.add_subplot(122, projection='3d')
                ax2.scatter(OD[:,0],OD[:,1],OD[:,2])

                plt.show()
                """

                ############ Step 2: Remove data with OD intensity less than β ############
                # remove transparent pixels (clear region with no tissue)
                ODhat = OD[~np.any(OD < beta, axis=1)] #Returns an array where OD values are above beta
                #Check by printing ODhat.min()

                ############# Step 3: Calculate SVD on the OD tuples ######################
                #Estimate covariance matrix of ODhat (transposed)
                # and then compute eigen values & eigenvectors.
                eigvals, eigvecs = np.linalg.eigh(np.cov(ODhat.T))


                ######## Step 4: Create plane from the SVD directions with two largest values ######
                #project on the plane spanned by the eigenvectors corresponding to the two 
                # largest eigenvalues    
                That = ODhat.dot(eigvecs[:,1:3]) #Dot product

                ############### Step 5: Project data onto the plane, and normalize to unit length ###########
                ############## Step 6: Calculate angle of each point wrt the first SVD direction ########
                #find the min and max vectors and project back to OD space
                phi = np.arctan2(That[:,1],That[:,0])

                minPhi = np.percentile(phi, alpha)
                maxPhi = np.percentile(phi, 100-alpha)

                vMin = eigvecs[:,1:3].dot(np.array([(np.cos(minPhi), np.sin(minPhi))]).T)
                vMax = eigvecs[:,1:3].dot(np.array([(np.cos(maxPhi), np.sin(maxPhi))]).T)


                # a heuristic to make the vector corresponding to hematoxylin first and the 
                # one corresponding to eosin second
                if vMin[0] > vMax[0]:    
                    HE = np.array((vMin[:,0], vMax[:,0])).T
                    
                else:
                    HE = np.array((vMax[:,0], vMin[:,0])).T


                # rows correspond to channels (RGB), columns to OD values
                Y = np.reshape(OD, (-1, 3)).T

                # determine concentrations of the individual stains
                C = np.linalg.lstsq(HE,Y, rcond=None)[0]

                # normalize stain concentrations
                maxC = np.array([np.percentile(C[0,:], 99), np.percentile(C[1,:],99)])
                tmp = np.divide(maxC,maxCRef)
                C2 = np.divide(C,tmp[:, np.newaxis])

                ###### Step 8: Convert extreme values back to OD space
                # recreate the normalized image using reference mixing matrix 

                #soltion for overflow by faysal
                exponent_values = -HERef.dot(C2)
                exponent_values = np.clip(exponent_values, -100, 100)  # Prevent extreme values
                Inorm = np.multiply(Io, np.exp(exponent_values))  

                #Inorm = np.multiply(Io, np.exp(-HERef.dot(C2))) #original code but exceed floating limit
                Inorm[Inorm>255] = 254
                Inorm = np.reshape(Inorm.T, (h, w, 3)).astype(np.uint8)  
                
                #clipping extreme value for H also
                H_exp = np.expand_dims(-HERef[:, 0], axis=1).dot(np.expand_dims(C2[0, :], axis=0))
                H_exp = np.clip(H_exp, -100, 100)  # Clip exponent values to avoid overflow
                H = np.multiply(Io, np.exp(H_exp))
                #H = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,0], axis=1).dot(np.expand_dims(C2[0,:], axis=0))))
                H[H>255] = 254
                H = np.reshape(H.T, (h, w, 3)).astype(np.uint8)

                # For E calculation
                E_exp = np.expand_dims(-HERef[:, 1], axis=1).dot(np.expand_dims(C2[1, :], axis=0))
                E_exp = np.clip(E_exp, -100, 100)  # Clip exponent values to avoid overflow
                E = np.multiply(Io, np.exp(E_exp))
                #E = np.multiply(Io, np.exp(np.expand_dims(-HERef[:,1], axis=1).dot(np.expand_dims(C2[1,:], axis=0))))
                E[E>255] = 254
                E = np.reshape(E.T, (h, w, 3)).astype(np.uint8)
                
                # Save the images
                base_filename = os.path.splitext(filename)[0]
                cv2.imwrite(os.path.join(output_dir, f"{base_filename}_normalized.jpg"), cv2.cvtColor(Inorm, cv2.COLOR_RGB2BGR))
                cv2.imwrite(os.path.join(h_folder, f"{base_filename}_H.jpg"), cv2.cvtColor(H, cv2.COLOR_RGB2BGR))
                cv2.imwrite(os.path.join(e_folder, f"{base_filename}_E.jpg"), cv2.cvtColor(E, cv2.COLOR_RGB2BGR))

                #print(f"Processed: {filename}")
            except Exception as e:
                # If an error occurs, save the original image and print the filename
                print(f"Skipping {filename} due to error: {e}")
                cv2.imwrite(os.path.join(output_dir, filename), cv2.imread(img_path))


In [ ]:
#H and E normalization for train data
input_folder = "blood Cancer/ALL/train/all_benign"
output_folder = "H and E blood Cancer/ALL/train/all_benign"
normalize_he_images(input_folder, output_folder, Io=240, alpha=1, beta=0.1)
print("Done")
input_folder = "blood Cancer/ALL/train/all_early"
output_folder = "H and E blood Cancer/ALL/train/all_early"
normalize_he_images(input_folder, output_folder, Io=240, alpha=1, beta=0.1)
print("Done")
input_folder = "blood Cancer/ALL/train/all_pre"
output_folder = "H and E blood Cancer/ALL/train/all_pre"
normalize_he_images(input_folder, output_folder, Io=240, alpha=1, beta=0.1)
print("Done")
input_folder = "blood Cancer/ALL/train/all_pro"
output_folder = "H and E blood Cancer/ALL/train/all_pro"
normalize_he_images(input_folder, output_folder, Io=240, alpha=1, beta=0.1)
print("Done")

Done
Done
Done
Done


In [ ]:
#H and E normalization for test data
input_folder = "blood Cancer/ALL/test/all_benign"
output_folder = "H and E blood Cancer/ALL/test/all_benign"
normalize_he_images(input_folder, output_folder, Io=240, alpha=1, beta=0.1)
print("Done")
input_folder = "blood Cancer/ALL/test/all_early"
output_folder = "H and E blood Cancer/ALL/test/all_early"
normalize_he_images(input_folder, output_folder, Io=240, alpha=1, beta=0.1)
print("Done")
input_folder = "blood Cancer/ALL/test/all_pre"
output_folder = "H and E blood Cancer/ALL/test/all_pre"
normalize_he_images(input_folder, output_folder, Io=240, alpha=1, beta=0.1)
print("Done")
input_folder = "blood Cancer/ALL/test/all_pro"
output_folder = "H and E blood Cancer/ALL/test/all_pro"
normalize_he_images(input_folder, output_folder, Io=240, alpha=1, beta=0.1)
print("Done")

Done
Done
Done
Done


> Below is the code for both vahadane and macenko method

In [2]:
import os
import cv2
import numpy as np
import staintools

def get_first_image(input_folder):
    """Get the first valid image from the dataset."""
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                return os.path.join(root, file)
    return None

def create_target_stain_matrix(image_path, normalizer_type):
    """Create target stain matrix from a single image."""
    normalizer = staintools.StainNormalizer(method=normalizer_type)
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Error reading the target stain image.")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    normalizer.fit(img)
    return normalizer

def normalize_dataset(input_folder, output_folder, stain_matrix_path=None, normalizer_type="vahadane"):
    """Normalize all images in the dataset using a single reference image."""
    if stain_matrix_path and os.path.exists(stain_matrix_path):
        normalizer = np.load(stain_matrix_path, allow_pickle=True).item()
    else:
        first_image = get_first_image(input_folder)
        if not first_image:
            raise ValueError("No images found in input directory")
        normalizer = create_target_stain_matrix(first_image, normalizer_type)
        if stain_matrix_path:
            np.save(stain_matrix_path, normalizer)
    
    for root, _, files in os.walk(input_folder):
        rel_path = os.path.relpath(root, input_folder)
        output_dir = os.path.join(output_folder, rel_path)
        os.makedirs(output_dir, exist_ok=True)
        
        for file in files:
            if not file.lower().endswith(('.png', '.jpg', '.jpeg')):
                continue
            
            input_path = os.path.join(root, file)
            output_path = os.path.join(output_dir, file)
            
            img = cv2.imread(input_path)
            if img is None:
                continue
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_normalized = normalizer.transform(img_rgb)
            cv2.imwrite(output_path, cv2.cvtColor(img_normalized, cv2.COLOR_RGB2BGR))


In [4]:
normalize_dataset(
    input_folder="./bloodcells_dataset",
    output_folder="./normalized_output of Blood Cancer vahadane",
    stain_matrix_path="./custom_stain_matrix_vahadane.npy",
    normalizer_type="vahadane"
)

In [3]:
normalize_dataset(
    input_folder="./bloodcells_dataset",
    output_folder="./normalized_output of Blood Cancer macenko",
    stain_matrix_path="./custom_stain_matrix_macenko.npy",
    normalizer_type="macenko"
)